In [ ]:
1. Опишите теоретическую модель градиентного бустинга

2. Напишите следующий год: сгенерируйте данные в виде набора точек; постройте регрессионную модель;
оцените её точность; визуализируйте результат.

3. Опишите метрику для проверки качества алгоритмов классификации.

4. Опишите технологию масштабирования признаков. Привидите примеры.

### 1 Номер

1) Создаем "базовую" модель, тренируем ее
2) Вычисляем ошибку, берем ее за инпут
3) Строим вторю модель "ансамбля" так, чтобы она училась предсказывать и минимизировать именно
ошибку
4) Повторяем так до тех пор пока не надоест

### 2 Номер 

Это буквально дз №3

### 3 Номер
``` python
print(f'Accuracy: {accuracy:.4f}')
# все верные предсказания / все предсказания
# сама по себе не очень полезна, особенно если датасет "странный", но позволяет быстро определить
# насколько все плохо  
print('Confusion Matrix:')
print(cm)
print("True Negative || False Positive")
print("False Negative || True Positive")
# Самая информативная метрика, от того и сложно читаемая
#  |                  | Predicted Positive | Predicted Negative |
#  |------------------|--------------------|--------------------|
#  | Actual Positive  | True Positive (TP) | False Negative (FN)|
#  | Actual Negative  | False Positive (FP)| True Negative (TN) |
# Показывает насколько хорошо модель справилась с классификацией. Вообще матрица может быть любого
# "квадратного" размера, но в таком случае ее исследовать становится тяжелее и тяжелее, поэтому 
# рассмотрим 2х2. ТР это когда значение = 1 и нейронка сказала что значение = 1. ТN соответственно 
# наоборот. FN и FP это когда нейронка ошибается в ту или иную сторону. По этой матрице можно 
# понять что вообще нейронка научилась делать. К примеру если вы изначально хотели научить ее 
# различать 10 классов, но так вышло что она может с хорошей точностью определять лишь 1, то 
# это будет заметно только в этой матрице, но не в accuracy 


print(f"Precision: {report['precision']:.4f}")
# это отношения количества реальных 1 к сумме всех предсказанных едениц TP/(TP+FP). Собственно по
# этой метрике можно как раз увидеть что 1 из 10 классов предсказывается очень хорошо, тогда как 
# общая точность плоха 
print(f"Recall: {report['recall']:.4f}")
# а тут отношение количества реальных едениц к сумме из предсказанных едениц и "недопредсказанных"
# едениц. Это нужно когда нам очень важно чтобы минимальное количество едениц осталось нулями 

print(f"F1-Score: {report['f1-score']:.4f}")
# а f1 как раз является средним гармоническим между присижном и реколлом, он нужен чтобы модель не 
# могла по хитрому нас обдурить, например всегда выбирая один из классов она максимизирует реколл
# или перетренировавшись на один из классов чтобы максимизировать присижн 
```

In [ ]:
1)